## I. Tris
### I. 1. Tri fusion

Tri en complexité optimale le plus simple à implémenter.

In [ ]:
let rec merge l1 l2 = 
  match (l1, l2) with
  | ([], _) -> l2
  | (_, []) -> l1
  | (h1::t1, h2::t2) -> if h1 < h2 then h1 :: merge t1 l2 else h2 :: merge l1 t2
;;

let rec split x y z =
  match x with
  | [] -> (y, z)
  | e :: x' -> split x' (e::z) y
;;

let rec tri_fusion l = 
  match l with
  | ([] | _::[]) -> l
  | _ -> let l1, l2 = split l [] [] in merge (tri_fusion l1) (tri_fusion l2)
;;

### I. 2. Tri rapide (implémentation mauvaise en pratique)

In [ ]:
(* Mauvaise implémentation en pratique, à la fois en temps (à cause de @) et en mémoire (structure persistante) *)
let rec quicksort l =
  match l with
  | ([] | _::[]) -> l
  | a::l' -> let inf = List.filter (fun x -> x<a) l' 
    and sup = List.filter (fun x -> x>a) l'
    in quicksort inf @ (a :: quicksort sup)

### I. 3. Tri à bulle
Sur place, stable, en $O(n^2)$. Tri de complexité quelconque le plus simple à implémenter.

In [ ]:
let swap t i j =
  let mem = t.(i) in
  t.(i) <- t.(j) ;
  t.(j) <- mem
;;

let tri_a_bulle tab = 
  let n = Array.length tab in
  for i = 0 to n-1 do
    for j = i+1 to n-1 do
      if t.(i) > t.(j) then swap t i j 
    done;
  done
;;

## II. Arbres binaires de recherche

Un arbre binaire de recherche est un arbre binaire étiqueté par un ensemble totalement ordonné, tel que pour tout noeud, les étiquettes des noeuds du sous-arbre gauche sont inférieures à l'étiquette du noeud, et les étiquettes des noeuds du sous-arbre droit sont supérieures à l'étiquette du noeud.

Cette structure sert notamment à implémenter des dictionnaires. Les opérations de base ont une complexité en $O(h)$, où $h$ est la hauteur de l'arbre.

In [ ]:
type tree =
  | F
  | N of tree * int * tree
;;

let rec recherche arbre el =
  match arbre with
  | F ->  false
  | N(g, a, d) -> if el = a then true 
                  else if el < a then recherche g el 
                  else recherche d el
;;

let rec ajout arbre el =
  match arbre with
  | F -> N(F, el, F)
  | N(g, a, d) -> if el < a then N(ajout g el, a, d) else N(g, a, ajout d el)
;;

let rec maxi arbre =
  match arbre with
  | F -> None
  | N(g, a, F) -> Some a
  | N(g, a, d) -> maxi d
;;

let rec suppression arbre el =
  match arbre with
  | F -> F
  | N(F, a, d) when a = el -> d
  | N(g, a, F) when a = el -> g
  | N(g, a, d) -> if el < a then N(suppression g el, a, d)
                  else if el > a then N(g, a, suppression d el)
                  else match maxi g with
                  | None -> failwith "erreur"
                  | Some y -> N(suppression g y, y, d)
;;

let rec suppression_bis arbre el =
  let rec extract_max arbre = 
    match arbre with
    | F -> failwith "erreur"
    | N(g, a, F) -> (g, a)
    | N(g, a, d) -> let (d', v) = extract_max d in (N(g, a, d'), v)
  in
  match arbre with
  | F -> F
  | N(F, a, d) when a = el -> d
  | N(g, a, F) when a = el -> g
  | N(g, a, d) -> if el < a then N(suppression_bis g el, a, d)
                  else if el > a then N(g, a, suppression_bis d el)
                  else (* a = el *)
                     let d', y = extract_max d in N(g, y, d')
;;

## III. Table de hachage
Structure de donnée impérative efficace pour implémenter un dictionnaire. Son implémentation telle que détaillée ici consiste en un `'a option array`. La fonction d'insertion fonctionne par **sondage linéaire** : elle examine les cases suivantes dans l'ordre en sélectionnant la première case vide. 

Une analyse de complexité amortie peut montrer que les opérations ont un coût constant en moyenne, pour un facteur de charge bien choisi.

In [ ]:
let est_present hashtbl hash el =
  let n = Array.length hashtbl in
  let i = ref (hash el) in
  let found = ref false in
  while !i < n && not !found do
    match hashtbl.(!i) with
    | None -> ()
    | Some a -> if a = el then found := true
  done ;
  !found
;;

let inserer hashtbl hash el =
  let i = ref (hash el) in
  let placed = ref false in
  while not !placed do
    match hashtbl.(!i) with
    | None -> hashtbl.(!i) <- Some el ; placed := true
    | Some _ -> i := !i + 1 mod Array.length hashtbl
  done
;;

let position hashtbl hash el =
  let n = Array.length hashtbl in
  let i = ref (hash el) in
  let pos = ref None in
  while !i < n && !pos <> None do
    match hashtbl.(!i) with
    | None -> ()
    | Some a -> if a = el then pos := Some !i
  done ;
  !pos
;;

let swap t i j =
  let mem = t.(i) in
  t.(i) <- t.(j) ;
  t.(j) <- mem
;;

let supprimer hashtbl hash el =
  let n = Array.length hashtbl in
  let pos = position hashtbl hash el in
  match pos with
  | None -> ()
  | Some p -> 
    hashtbl.(p) <- None ;

    let rec decaler i =
      (* 'decaler' suppose que hashtbl.(i-1 mod n) = None et que 0 <= i < n *)
      match hashtbl.(i) with
      | None -> ()
      | Some v ->
        if hash v <> i then begin
          swap hashtbl i (i - 1 mod n) ; 
          decaler (i + 1 mod n)
        end
    in
    decaler (p + 1 mod n)
;;

## IV. File de priorité et tas

Un tas est un arbre tournoi quasi-complet à gauche. Tournoi : chaque étiquette de chaque noeud est inférieure à celle de ses enfants.  Complexité logarithmique dans le pire cas. 

In [ ]:
type 'a tas = {mutable taille : int ; contenu : (int * 'a) array } 

let tas_vide capacite el = { taille = 0 ; contenu = Array.make capacite el };;

let est_vide tas = tas.taille = 0;

let swap t i j =
  let mem = t.(i) in
  t.(i) <- t.(j) ;
  t.(j) <- mem
;;

let rec remontee tab pos = 
  if pos > 0 then 
    let pere = (pos + 1)/2 - 1 in 
    if tab.(pere) > tab.(pos) then begin
      swap tab pere pos;
      remontee tab pere
    end
;;

let rec 

## V. File

In [ ]:
type 'a queue = {tab : 'a array ; mutable deb : int ; mutable fin : int ; mutable vide : bool};;

let vide cap el_init = {tab = Array.make cap el_init ; deb = 0 ; fin = 0 ; vide = true};;

let est_vide q = q.vide;;

let insertion q el = 
  if q.fin mod Array.length q.tab = q.deb && (not q.vide) then failwith "queue pleine"
  else begin
    q.tab.(q.fin) <- el ;
    q.fin <- q.fin + 1 mod Array.length q.tab ;
    q.vide <- false
  end
;;

let suppression q el =
  if q.vide then failwith "queue vide"
  else begin
    q.deb <- q.deb + 1 mod Array.length q.tab ;
    if q.fin = q.deb then q.vide <- true
  end
;;

## VI. Parcours de graphes et applications

## VII. Plus court chemin dans un graphe
### VII. 1. Algorithme de Floyd-Warshall

### VII. 2. Algorithme de Dijkstra